In [1]:
using LightGraphs

"""
inverse logit function
"""
invLogit(x) = 1./(1.+e.^-x)   

"""
given graph and probability, adds a node which must have >0
connections by flipping biased coin for each existing node
"""
function addNode2(graph, p)
    add_vertex!(graph)
    x = nv(graph)
    degree = 0
    while degree ==0
        flips = rand(x-1)
        for i = 1:x-1
            if p[i]>flips[i]
                add_edge!(graph,i,x)
                degree +=1 
            end
        end
    end
    return graph
end


"""
given graph, b vector, and a_0, adds a new node as specifiec by the model
"""
function addPrefNode(g,b,a_0 = -7)
    n = nv(g)
    L = laplacian_matrix(g)
    a = lufact(L) \ (b - mean(b))    
    p = invLogit(a+a_0)
    addNode2(g,p)
    push!(b,0)
    return g
end


"""
given graph and number of new edges desired, randomly adds edges between existing nodes
"""
function randEdgeGen(graph, newedges)
    for i in 1:newedges
        z = newedges
        x = collect(1:nv(graph))
        edge1 = rand(x)
        deleteat!(x, edge1)
        edge2 = rand(x)
        add_edge!(graph,edge1,edge2)
    end
    return graph
end
;

In [2]:
"""
soft threshold
"""
soft(c,lambda) = sign(c).*max(abs(c)-lambda/2,0)

"""
computes gradient
"""
function gradient2(a,a_0,u,L,rho,b,y)
    grad = -1.*(y-invLogit(a+a_0))+L*u + rho*L*(L*a-b)
    return grad
end;


"""
computes hessian
"""
function hessian(a,a_0,rho,L)
    hess = Diagonal(vec((invLogit(a+a_0).*(1-invLogit(a+a_0)))))+rho*L^2
    return hess
end;




"""
newton raphson for a update
"""
function newton(y_i,a_0,L,rho,b,u)
    a = zeros(length(y_i),1)
    a_old = a
    iters = 0
    diff = 1.0
    while(diff >STOP_DIFF && iters< MAX_ITER )
        grad = gradient2(a_old,a_0,u,L,rho,b,y_i)
        hess = hessian(a_old,a_0, rho,L)
        a = a_old - inv(hess)*grad
        diff = norm(a-a_old)
        a_old = a
        iters = iters+1
    end
    if(iters == MAX_ITER)
        print("max iter reached")
    end
    return a
end
;




In [3]:
levels = 10     #number of levels in binary tree
g = BinaryTree(levels)
n = nv(g)
b = (rand(n) .< 8 / n)*1. 
genb = copy(b)  # save for later
g = randEdgeGen(g,1000)
A = Array{Int64,2}[]
L =  SparseMatrixCSC{Int64,Int64}[]
push!(L, laplacian_matrix(g))
numnewnodes = 2
a_0 = -5
# creates matrix A and L where A[i] is the connections for ith node and L[i] is the laplacian of the i-1st time step 
for i in 1:numnewnodes  
    g = addPrefNode(g,b, a_0)
    push!(L,laplacian_matrix(g))
    connects = zeros(2^levels-2+i,1)  #-1 for -1 1 coding
    connects[neighbors(g,nv(g))] = 1
    push!(A,connects)
end


t = 2^levels-1+numnewnodes #number of nodes at time t
t_0 = 2^levels-1  # number of initial nodes
;

const MAX_ITER = 1000
const STOP_DIFF = 0.001;

rho = 1.5
lambda = 1.1
new = numnewnodes
a = zeros(t-1,new)
b = zeros(t_0)
u = zeros(t-1,new)
#	alpha = 1.5  #relaxation parameter
iters = 0
diff = 1.0
b_old = b;


Works on one node

In [5]:
	for i in 1:new    
   		a[1:(t_0 +i-1),i]= newton(A[i],a_0,L[i],rho,[b' zeros(i-1)']',u[1:(t_0 +i-1),i])
   	end

In [7]:
    c = zeros(t-1)
    for i in 1:new
    	c[1:(t_0 +i-1)] = c[1:(t_0 +i-1)]+ u[1:(t_0 +i-1),i]+rho*(L[i]*a[1:(t_0 +i-1),i])/(rho)
    end

In [12]:
b = soft(c[1:t_0],lambda)

1023-element Array{Float64,1}:
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
  ⋮  
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
  0.0
 -0.0

In [ ]:
@time begin

for j in 1:50 
	for i in 1:new    
   		a[1:(t_0 +i-1),i]= newton(A[i],a_0,L[i],rho,[b' zeros(i-1)']',u[1:(t_0 +i-1),i])
   	end
        
    #b update
    c = zeros(t-1)
    for i in 1:new
    	c[1:(t_0 +i-1)] = c[1:(t_0 +i-1)]+ u[1:(t_0 +i-1),i]+rho*(L[i]*a[1:(t_0 +i-1),i])/(rho)
    end
    b = soft(c[1:t_0],lambda)
    for i in 1:new
    	u[1:(t_0 +i-1),i] = u[1:(t_0 +i-1),i]+ rho*(L[i]*a[1:(t_0 +i-1),i]-[b' zeros(i-1)']')
    end
    diff  = norm(b-b_old)
    b_old = b
    println(diff)
end


end


In [ ]:
@time begin

for j in 1:50     
    for i in 1:new
        a[1:length(A[i]),i] = newton(A[i],a_0,L[i],rho,[b' zeros(i-1)']',u[1:length(A[i]),i])
    end
    #b update
    c = zeros(t-1)
    for i in 1:new
        c[1:size(L[i])[1]] = c[1:size(L[i])[1]]+ u[1:size(L[i])[1],i]+rho*(L[i]*a[1:size(L[i])[1],i])/(rho)
    end
    b = soft(c[1:t_0],lambda)
    for i in 1:new
        u[1:length(A[i]),i] = u[1:length(A[i]),i]+ rho*(L[i]*a[1:length(A[i]),i]-[b' zeros(i-1)']')
    end
    diff  = norm(b-b_old)
    b_old = b
    println(diff)
end
    
end

In [ ]:
sort(b[:,1])

Ver 2

In [4]:
@time begin

for j in 1:50 
	for i in 1:new    
   		a[1:(t_0 +i-1),i]= newton(A[i],a_0,L[i],rho,[b' zeros(i-1)']',u[1:(t_0 +i-1),i])
   	end
        
    #b update
    c = zeros(t-1)
    for i in 1:new
    	c[1:(t_0 +i-1)] = c[1:(t_0 +i-1)]+ u[1:(t_0 +i-1),i]+rho*(L[i]*a[1:(t_0 +i-1),i])/(rho)
    end
    b = soft(c[1:t_0],lambda)
    for i in 1:new
    	u[1:(t_0 +i-1),i] = u[1:(t_0 +i-1),i]+ rho*(L[i]*a[1:(t_0 +i-1),i]-[b' zeros(i-1)']')
    end
    diff  = norm(b-b_old)
    b_old = b
    println(diff)
end


end


0.40905792469445834
1.13512143925225
1.2218802351081564
1.651122065049837
1.699965377383836
2.324091756618478
3.313895503988496
5.021249951823244
7.526157522850481
11.781672377060174
18.731704534170362
30.08977671158003
48.53797775884829
78.3990445018065
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
 62.939256 seconds (3.90 M allocations: 9.832 GB, 0.92% gc time)


Ver 3

In [16]:
b

1024-element Array{Float64,1}:
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
   ⋮
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN
 NaN

In [4]:
@time begin

b = zeros(t-1)
b_old = b
for j in 1:50 
	for i in 1:new    
   		a[1:(t_0 +i-1),i]= newton(A[i],a_0,L[i],rho,b[1:(t_0 +i-1)],u[1:(t_0 +i-1),i])
   	end
        
    #b update
    c = zeros(t-1)
    for i in 1:new
    	c[1:(t_0 +i-1)] = c[1:(t_0 +i-1)]+ u[1:(t_0 +i-1),i]+rho*(L[i]*a[1:(t_0 +i-1),i])/(rho)
    end
    b = soft(c,lambda)
    for i in 1:new
    	u[1:(t_0 +i-1),i] = u[1:(t_0 +i-1),i]+ rho*(L[i]*a[1:(t_0 +i-1),i]-b[1:(t_0 +i-1)])
    end
    diff  = norm(b-b_old)
    b_old = b
    println(diff)
end

@time begin
for j in 1:1000     
    a= newton(A,a_0,L,rho,b,u)
    #b update
    c = zeros(t-1)
    c = c+ u+rho*(L*a)/(rho)
    b = soft(c,lambda)
    u = u+ rho*(L*a-b)
    diff  = norm(b-b_old)
    b_old = b
    println(diff)
end
end
end


0.39972889715465487
0.8970037380519437
1.1169359311775575
1.4258072810395157
1.43045619297965
2.0681113087027096
3.155210940541284
4.886839079035107
7.545563565666765
11.933141719699192
19.021936096398168
30.87925494257135
49.765330081182945
80.62596547366235
130.59766486144912
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
129.849989 seconds (6.44 M allocations: 19.489 GB, 0.63% gc time)
